<a href="https://colab.research.google.com/github/mkielo3/mammalian_brains/blob/main/quickstart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install -q numpy pandas scipy scikit-learn matplotlib seaborn pillow ipykernel torch torchvision torchaudio altair datasets numba numbasom==0.0.5 tensorboard pysal vl-convert-python

!git clone https://github.com/mkielo3/mammalian_brains.git
%cd mammalian_brains

In [1]:
import torch
import pandas as pd

from tqdm import tqdm
from numbasom.core import lattice_closest_vectors
from analysis.som import SOM
from utils import save_output_to_pickle
from models.olfaction.olfaction import Olfaction
from models.vision.vision import Vision
from models.audio.audio import Audio
from models.touch.touch import Touch
from models.memory.memory import Memory
import argparse

from config import Args
args = Args()


fast = True
args.som_size = (5,5) if fast else (25, 25)
args.experiment_name = "main_results_fast" if fast else "main_results"
modality_list = [Olfaction(args), Vision(args), Audio(args), Touch(args), Memory(args)]

/Users/matthewkielo/miniforge3/envs/babylm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
for modality in modality_list:
	print ("\n", modality.modality, pd.Timestamp.now())
	if modality.modality == 'audio':
		args.setup_models = True # always setup, bc weights too large to save in repo
	else:
		args.setup_models = False # otherwise setup with setup.py

	# 1. Train/Download Model
	modality.setup_model()
	modality.setup_som()

	# 2. Get activations for each patch
	patches = modality.get_patches()
	activation_list = []
	for p in tqdm(patches):
		p, static = modality.generate_static(p)
		activation = modality.calculate_activations(static)
		activation_list.append([p, activation])

	# 3. Fit SOM
	x_mat = torch.stack([x[1] for x in activation_list]).numpy()
	som = modality.initialize_som(SOM)
	lattice = som.train(x_mat, num_iterations=args.som_epochs, initialize=args.som_init, normalize=False, start_lrate=args.som_lr)
	
	# 4. Get coordinates for each BMU
	coordinate_list = [x[0] for x in activation_list]
	closest = lattice_closest_vectors(x_mat, lattice, additional_list=coordinate_list)

	# 5. Save
	output = {"closest": closest, 
			"coord_map": coordinate_list,
			"x_range": (0, max([x[0][0] for x in activation_list])),
			"y_range": (0, max([x[0][1] for x in activation_list])),
			"lattice": lattice,
			"som": None,
			"samples": modality.sample_data,
			"modality": modality.modality,
			"args": args,
			"activations": activation_list}

	save_output_to_pickle(output, args.experiment_name)


 olfaction 2025-02-11 09:44:48.429016


100%|██████████| 4096/4096 [00:01<00:00, 3286.38it/s]


Initializing SOM with Random
Done Init
SOM training took: 1.153494 seconds.
Finding closest data points took: 0.082444 seconds.
Saved output to: output/main_results_fast/olfaction.pkl

 vision 2025-02-11 09:44:51.074540


Using cache found in /Users/matthewkielo/.cache/torch/hub/pytorch_vision_v0.10.0
/Users/matthewkielo/miniforge3/envs/babylm/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/matthewkielo/miniforge3/envs/babylm/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
100%|██████████| 5625/5625 [01:42<00:00, 55.12it/s]


Initializing SOM with Random
Done Init
SOM training took: 74.167695 seconds.
Finding closest data points took: 3.509871 seconds.
Saved output to: output/main_results_fast/vision.pkl

 audio 2025-02-11 09:47:53.692764


...

Initializing SOM with Random
Done Init
SOM training took: 51.615221 seconds.
Finding closest data points took: 0.911024 seconds.
Saved output to: output/main_results_fast/audio.pkl

 touch 2025-02-11 09:55:14.132688
[TouchDataset] Refreshing tuples...
Loaded "ObjectClusterDataset" - split "test" with 16119 records...
[TouchDataset] Refreshing tuples...
Base LR = 1.000000e-03
tensor(0.5800)


100%|██████████| 548/548 [00:00<00:00, 915.95it/s]


Initializing SOM with Random
Done Init
SOM training took: 21.150975 seconds.
Finding closest data points took: 0.107233 seconds.
Saved output to: output/main_results_fast/touch.pkl

 memory 2025-02-11 09:55:38.581212


100%|██████████| 4455/4455 [00:00<00:00, 270967.16it/s]


Initializing SOM with Random
Done Init
SOM training took: 0.869863 seconds.
Finding closest data points took: 0.033581 seconds.
Saved output to: output/main_results_fast/memory.pkl


In [ ]:
for modality in modality_list:
	break

In [13]:
from utils import load_pickle_output

In [18]:
import pickle

def load_pickle_output(experiment_name, modality):
   filename = f"output/{experiment_name}/{modality}.pkl"
   with open(filename, 'rb') as f:
       output = pickle.load(f)
   return output

data = load_pickle_output(args.experiment_name, modality.modality)

In [19]:
data

{'closest': {(0, 0): [(3160, 0)],
  (0, 1): [(3830, 0)],
  (0, 2): [(3996, 0)],
  (0, 3): [(206, 0)],
  (0, 4): [(1393, 0)],
  (1, 0): [(3305, 0)],
  (1, 1): [(3687, 0)],
  (1, 2): [(3179, 0)],
  (1, 3): [(934, 0)],
  (1, 4): [(472, 0)],
  (2, 0): [(3305, 0)],
  (2, 1): [(1242, 0)],
  (2, 2): [(394, 0)],
  (2, 3): [(107, 0)],
  (2, 4): [(503, 0)],
  (3, 0): [(1345, 0)],
  (3, 1): [(1859, 0)],
  (3, 2): [(3619, 0)],
  (3, 3): [(1047, 0)],
  (3, 4): [(1047, 0)],
  (4, 0): [(848, 0)],
  (4, 1): [(848, 0)],
  (4, 2): [(1201, 0)],
  (4, 3): [(1047, 0)],
  (4, 4): [(1047, 0)]},
 'coord_map': [(0, 0),
  (1, 0),
  (2, 0),
  (3, 0),
  (4, 0),
  (5, 0),
  (6, 0),
  (7, 0),
  (8, 0),
  (9, 0),
  (10, 0),
  (11, 0),
  (12, 0),
  (13, 0),
  (14, 0),
  (15, 0),
  (16, 0),
  (17, 0),
  (18, 0),
  (19, 0),
  (20, 0),
  (21, 0),
  (22, 0),
  (23, 0),
  (24, 0),
  (25, 0),
  (26, 0),
  (27, 0),
  (28, 0),
  (29, 0),
  (30, 0),
  (31, 0),
  (32, 0),
  (33, 0),
  (34, 0),
  (35, 0),
  (36, 0),
  (37, 0),


In [21]:
from analysis.receptive_field import calculate_receptive_fields

In [3]:
from utils import save_som_plot
save_som_plot(args.experiment_name, modality_list, args)

NameError: name 'args' is not defined